In [38]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# sqrtm

> Methods for computing sqrt of a matrix

In [39]:
#| default_exp sqrtm

In [40]:
#| hide
from nbdev.showdoc import *

In [41]:
#| export
import torch
from torch.autograd import Function, Variable

## Steve Li's method

In [42]:
#| export

use_li = True  # come back and turn this on if you want to see/use the full code

if use_li:      # lighten the load of imports since we won't use li's in production
    import numpy as np
    import scipy.linalg

In [53]:
#| export
class MatrixSquareRoot_li(Function):
    """
    From https://github.com/steveli/pytorch-sqrtm/blob/master/sqrtm.py, which sadly does not install as a package.  LICENSE included below
    Square root of a positive definite matrix.

    NOTE: matrix square root is not differentiable for matrices with
          zero eigenvalues.
    """
    @staticmethod
    def forward(ctx, input):
        m = input.detach().cpu().numpy().astype(np.float_)  # SHH: note how this immediately switches to CPU & numpy :-( 
        sqrtm = torch.from_numpy(scipy.linalg.sqrtm(m).real).to(input)
        ctx.save_for_backward(sqrtm)
        return sqrtm

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = None
        if ctx.needs_input_grad[0]:
            sqrtm, = ctx.saved_tensors
            sqrtm = sqrtm.data.cpu().numpy().astype(np.float_)
            gm = grad_output.data.cpu().numpy().astype(np.float_)

            # Given a positive semi-definite matrix X,
            # since X = X^{1/2}X^{1/2}, we can compute the gradient of the
            # matrix square root dX^{1/2} by solving the Sylvester equation:
            # dX = (d(X^{1/2})X^{1/2} + X^{1/2}(dX^{1/2}).
            grad_sqrtm = scipy.linalg.solve_sylvester(sqrtm, sqrtm, gm)

            grad_input = torch.from_numpy(grad_sqrtm).to(grad_output)
        return grad_input


sqrtm_li = MatrixSquareRoot_li.apply


LICENCE_LI = """
MIT License

Copyright (c) 2022 Steven Cheng-Xian Li

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

Steve Li's test code for the above:


In [85]:
#| eval: false 
from torch.autograd import gradcheck

In [90]:
#| eval: false 
if use_li:
    k = torch.randn(1000, 1000).double()
    # Create a positive definite matrix
    pd_mat = (k.t().matmul(k)).requires_grad_()
    with torch.no_grad():
        sq = sqrtm_li(pd_mat)
    print("sq =\n",sq)
    #print("Running gradcheck...")
    #test = gradcheck(sqrtm_li, (pd_mat,))
    #print(test)

sq =
 tensor([[ 2.6360e+01, -5.2896e-01,  4.6020e-01,  ...,  4.6385e-01,
         -2.5534e-01,  3.3804e-01],
        [-5.2896e-01,  2.5773e+01, -7.2415e-01,  ..., -2.6621e-02,
          3.0918e-01, -7.8089e-02],
        [ 4.6020e-01, -7.2415e-01,  2.5863e+01,  ..., -5.2346e-01,
          1.4617e-01, -2.5943e-01],
        ...,
        [ 4.6385e-01, -2.6621e-02, -5.2346e-01,  ...,  2.6959e+01,
          3.6158e-01, -4.6653e-01],
        [-2.5534e-01,  3.0918e-01,  1.4617e-01,  ...,  3.6158e-01,
          2.6692e+01, -4.4417e-01],
        [ 3.3804e-01, -7.8089e-02, -2.5943e-01,  ..., -4.6653e-01,
         -4.4417e-01,  2.8916e+01]], dtype=torch.float64)


## Subhransu Maji's method(s)

From https://github.com/msubhransu/matrix-sqrt

In [123]:
#| export

# Compute error
def compute_error(A, sA):
    normA = torch.sqrt(torch.sum(torch.sum(A * A, dim=1),dim=1))
    error = A - torch.bmm(sA, sA)
    error = torch.sqrt((error * error).sum(dim=1).sum(dim=1)) / normA
    return torch.mean(error)


def sqrt_newton_schulz_autograd(A, 
                                numIters=20, # found experimentally by SHH, comparing w/ Li's method
                                calc_error=False,):
    """Modified from from https://people.cs.umass.edu/~smaji/projects/matrix-sqrt/
    "The drawback of the autograd approach [i.e., this approach] is that a naive implementation stores all the intermediate results. 
    Thus the memory overhead scales linearly with the number of iterations which is problematic for large matrices."
    """
    if len(A.data.shape) < 3: A = A.unsqueeze(0)
    batchSize, dim1, dim2 = A.data.shape
    assert dim1==dim2
    dim, dtype, device = dim1, A.dtype, A.device
    normA = A.mul(A).sum(dim=1).sum(dim=1).sqrt()
    Y = A.div(normA.view(batchSize, 1, 1).expand_as(A));
    I = Variable(torch.eye(dim,dim, device=device).view(1, dim, dim).
                 repeat(batchSize,1,1).type(dtype),requires_grad=False)
    Z = Variable(torch.eye(dim,dim, device=device).view(1, dim, dim).
                 repeat(batchSize,1,1).type(dtype),requires_grad=False)

    for i in range(numIters):
        T = 0.5*(3.0*I - Z.bmm(Y))
        Y = Y.bmm(T)
        Z = T.bmm(Z)
    
    sA = Y*torch.sqrt(normA).view(batchSize, 1, 1).expand_as(A)
    if calc_error:
        error = compute_error(A, sA)
        return sA, error
    return sA


def sqrt_newton_schulz(A,               # matrix to be sqrt-ified
                       numIters=20,      # numIters=7 found via experimentation
                       calc_error=False, # setting False disables Maji's error reporting
                      ):
    """
    Sqrt of matrix via Newton-Schulz algorithm
    Modified from https://github.com/msubhransu/matrix-sqrt/blob/cc2289a3ed7042b8dbacd53ce8a34da1f814ed2f/matrix_sqrt.py#LL72C1-L87C19
    # Forward via Newton-Schulz iterations (non autograd version)
    # Seems to be slighlty faster and has much lower memory overhead
    
    ... Original code didn't preserve device, had no batch dim checking -SHH
    """
    while len(A.data.shape) < 3:  # needs a batch dimension
        A = A.unsqueeze(0)
    batchSize, dim1, dim2 = A.data.shape
    assert dim1==dim2
    dim, dtype, device = dim1, A.dtype, A.device
    normA = A.mul(A).sum(dim=1).sum(dim=1).sqrt()
    Y = A.div(normA.view(batchSize, 1, 1).expand_as(A));
    I = torch.eye(dim,dim, device=device, dtype=dtype).view(1, dim, dim).repeat(batchSize,1,1)
    Z = torch.eye(dim,dim, device=device, dtype=dtype).view(1, dim, dim).repeat(batchSize,1,1)
    for i in range(numIters):
        T = 0.5*(3.0*I - Z.bmm(Y))
        Y = Y.bmm(T)
        Z = T.bmm(Z)
        
    sA = Y*torch.sqrt(normA).view(batchSize, 1, 1).expand_as(A)
    if calc_error: 
        error = compute_error(A, sA)
        return sA, error
    else:
        return sA

    
""" 
# Only used if backprop needed, which it isn't for FAD.  Leaving it here anyway. -SHH
def lyap_newton_schulz(z, dldz, numIters, dtype):
    # Backward via iterative Lyapunov solver.
    batchSize = z.shape[0]
    dim = z.shape[1]
    normz = z.mul(z).sum(dim=1).sum(dim=1).sqrt()
    a = z.div(normz.view(batchSize, 1, 1).expand_as(z))
    I = torch.eye(dim,dim).view(1, dim, dim).repeat(batchSize,1,1).type(dtype)
    q = dldz.div(normz.view(batchSize, 1, 1).expand_as(z))
    for i in range(numIters):
        q = 0.5*(q.bmm(3.0*I - a.bmm(a)) - a.transpose(1, 2).bmm(a.transpose(1,2).bmm(q) - q.bmm(a)) )
        a = 0.5*a.bmm(3.0*I - a.bmm(a))
    dlda = 0.5*q
    return dlda
"""


LICENCE_SM = """
MIT License

Copyright (c) 2017 Subhransu Maji

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

## Error tests

In [104]:
#| eval: false 
sa1, error = sqrt_newton_schulz_autograd( pd_mat.unsqueeze(0), numIters=20, calc_error=True ) 
print("sa1 =\n",sa1)
print("error =",error.detach().item())

sa1 =
 tensor([[[ 2.6360e+01, -5.2896e-01,  4.6021e-01,  ...,  4.6384e-01,
          -2.5535e-01,  3.3805e-01],
         [-5.2896e-01,  2.5773e+01, -7.2415e-01,  ..., -2.6632e-02,
           3.0917e-01, -7.8092e-02],
         [ 4.6021e-01, -7.2415e-01,  2.5863e+01,  ..., -5.2344e-01,
           1.4618e-01, -2.5943e-01],
         ...,
         [ 4.6384e-01, -2.6632e-02, -5.2344e-01,  ...,  2.6959e+01,
           3.6150e-01, -4.6653e-01],
         [-2.5535e-01,  3.0917e-01,  1.4618e-01,  ...,  3.6150e-01,
           2.6692e+01, -4.4418e-01],
         [ 3.3805e-01, -7.8092e-02, -2.5943e-01,  ..., -4.6653e-01,
          -4.4418e-01,  2.8916e+01]]], dtype=torch.float64,
       grad_fn=<MulBackward0>)
error = 4.759428080865442e-08


In [105]:
#| eval: false 
sa2, error = sqrt_newton_schulz( pd_mat.unsqueeze(0), numIters=20, calc_error=True ) 
print("sa2 =\n",sa2)
print("error =",error.detach().item())

sa2 =
 tensor([[[ 2.6360e+01, -5.2896e-01,  4.6021e-01,  ...,  4.6384e-01,
          -2.5535e-01,  3.3805e-01],
         [-5.2896e-01,  2.5773e+01, -7.2415e-01,  ..., -2.6632e-02,
           3.0917e-01, -7.8092e-02],
         [ 4.6021e-01, -7.2415e-01,  2.5863e+01,  ..., -5.2344e-01,
           1.4618e-01, -2.5943e-01],
         ...,
         [ 4.6384e-01, -2.6632e-02, -5.2344e-01,  ...,  2.6959e+01,
           3.6150e-01, -4.6653e-01],
         [-2.5535e-01,  3.0917e-01,  1.4618e-01,  ...,  3.6150e-01,
           2.6692e+01, -4.4418e-01],
         [ 3.3805e-01, -7.8092e-02, -2.5943e-01,  ..., -4.6653e-01,
          -4.4418e-01,  2.8916e+01]]], dtype=torch.float64,
       grad_fn=<MulBackward0>)
error = 4.759428080865442e-08


In [106]:
#| eval: false 
if use_li:
    diff = sa1 - sq
    print("diff = \n",diff) 

diff = 
 tensor([[[-3.7835e-05,  3.8437e-07,  8.7208e-06,  ..., -1.2816e-05,
          -1.8744e-05,  1.3386e-05],
         [ 3.8437e-07, -1.9896e-06,  2.2977e-06,  ..., -1.1004e-05,
          -1.0735e-05, -3.3250e-06],
         [ 8.7208e-06,  2.2977e-06, -7.5329e-06,  ...,  2.2518e-05,
           1.9394e-05, -3.9655e-06],
         ...,
         [-1.2816e-05, -1.1004e-05,  2.2518e-05,  ..., -8.1416e-05,
          -7.1274e-05, -1.4799e-06],
         [-1.8744e-05, -1.0735e-05,  1.9394e-05,  ..., -7.1274e-05,
          -8.0163e-05, -1.6137e-05],
         [ 1.3386e-05, -3.3250e-06, -3.9655e-06,  ..., -1.4799e-06,
          -1.6137e-05, -2.6324e-05]]], dtype=torch.float64,
       grad_fn=<SubBackward0>)


In [107]:
#| eval: false 
if use_li:
    diff = sa2 - sq
    print("diff = \n",diff) 

diff = 
 tensor([[[-3.7835e-05,  3.8437e-07,  8.7208e-06,  ..., -1.2816e-05,
          -1.8744e-05,  1.3386e-05],
         [ 3.8437e-07, -1.9896e-06,  2.2977e-06,  ..., -1.1004e-05,
          -1.0735e-05, -3.3250e-06],
         [ 8.7208e-06,  2.2977e-06, -7.5329e-06,  ...,  2.2518e-05,
           1.9394e-05, -3.9655e-06],
         ...,
         [-1.2816e-05, -1.1004e-05,  2.2518e-05,  ..., -8.1416e-05,
          -7.1274e-05, -1.4799e-06],
         [-1.8744e-05, -1.0735e-05,  1.9394e-05,  ..., -7.1274e-05,
          -8.0163e-05, -1.6137e-05],
         [ 1.3386e-05, -3.3250e-06, -3.9655e-06,  ..., -1.4799e-06,
          -1.6137e-05, -2.6324e-05]]], dtype=torch.float64,
       grad_fn=<SubBackward0>)


## Speed & device tests

In [108]:
#| eval: false 
from aeiou.core import get_device

In [109]:
#| eval: false 
device = get_device()
print('device = ',device)
n,m = 1000, 1000
with torch.no_grad(): 
    k = torch.randn(n, m, device=device)
    pd_mat2 = (k.t().matmul(k)) # Create a positive definite matrix, no grad

device =  cuda


In [117]:
#| eval: false 
# %%timeit
if use_li:
    sq2 = sqrtm_li(pd_mat2)

Result of `%%timeit`:

`1.12 s ± 191 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)`

In [118]:
sq2

tensor([[ 2.6228e+01, -5.3979e-01, -7.5547e-01,  ...,  7.5279e-01,
          3.8927e-02, -3.3051e-01],
        [-5.3979e-01,  2.6421e+01,  2.0006e-01,  ..., -6.1695e-01,
          2.1587e-01, -1.6249e-01],
        [-7.5547e-01,  2.0006e-01,  2.7266e+01,  ...,  3.0650e-01,
          2.9098e-02, -2.5314e-02],
        ...,
        [ 7.5279e-01, -6.1695e-01,  3.0650e-01,  ...,  2.6963e+01,
         -5.9428e-01, -6.7507e-02],
        [ 3.8927e-02,  2.1587e-01,  2.9098e-02,  ..., -5.9428e-01,
          2.7010e+01, -6.8757e-01],
        [-3.3051e-01, -1.6249e-01, -2.5314e-02,  ..., -6.7507e-02,
         -6.8757e-01,  2.7621e+01]], device='cuda:0')

In [120]:
#| eval: false 
# %%timeit
sq3 = sqrt_newton_schulz(pd_mat2.unsqueeze(0), numIters=20)[0]

Result of `%%timeit`:

`8.8 ms ± 23.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)`

## Wrapper around our method of choice:
TLDR, we'll use Maji's Newton-Schulz method.  Newton-Schulz is an approximate iterative method rather than an exact matrix sqrt, however, with 7 iterations the error is below 1e-5, (presumably significantly) lower than other errors in the problem.  

In [124]:
#|export 
def sqrtm(A, method='maji', numIters=20):
    "wrapper function for matrix sqrt algorithm of choice. Also we'll turn off all gradients"
    with torch.no_grad():
        if method=='maji':
            return sqrt_newton_schulz(A, numIters=numIters, calc_error=False).squeeze()  # get rid of any useless batch dimensions
        elif method=='li': 
            return sqrtm_li(A)
        else:
            raise ValueError(f"Invalid method: {method}") 

In [125]:
#| eval: false  
sqrtm(pd_mat2, method='maji') - sqrtm(pd_mat2, method='li') 

tensor([[ 1.9073e-06,  1.5616e-05,  9.0003e-06,  ..., -8.9407e-07,
         -3.7104e-05,  6.7353e-06],
        [ 1.9193e-05, -6.1035e-05, -4.7386e-06,  ...,  7.8678e-06,
          2.9802e-05, -5.8711e-06],
        [ 7.5698e-06, -5.4836e-06, -3.8147e-05,  ...,  2.0236e-05,
          1.6876e-06, -2.9024e-05],
        ...,
        [-3.3975e-06,  3.6955e-06,  2.2471e-05,  ..., -1.1444e-05,
          3.6955e-06,  1.2971e-05],
        [-3.6355e-05,  2.5883e-05,  7.5437e-06,  ...,  9.4771e-06,
         -6.8665e-05,  6.7949e-06],
        [ 4.2319e-06, -1.3113e-05, -2.6686e-05,  ...,  9.9763e-06,
          1.1921e-05,  5.7220e-06]], device='cuda:0')

In [127]:
#| hide
import nbdev; nbdev.nbdev_export()